In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [81]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\ayeh21\\Documents\\HesBO\\baseline\\test_fn.py'>

# Algorithm 1: Shift or twist subspace to best performing points

In [82]:
importlib.reload(test_fn)
fn = test_fn.Ackley()

low_dim = 2
fn_dim = 200

x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [83]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood

In [84]:
x.shape

torch.Size([100, 2])

In [85]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

In [86]:
gp = SingleTaskGP(x, y.reshape(-1,1) * -1)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [87]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [88]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2) - 1, torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=10, raw_samples=20,
)

In [89]:
candidate

tensor([[-0.4599, -0.4582]])

In [90]:
acq_value

tensor(2.8326)

In [91]:
fn.evaluate(torch.tensor([1,1]))

tensor([3.6254])

In [92]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, 
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = True,
                                   twist_step_size = 1,
                                   move_subspace = False,
                                   move_step_size = 1,
                                   restarts = 0,
                                   samples = 200, max_iter = 1000, 
                                   initial_offset = 2, trace = False):
    
    best_found = 1e10
    A = torch.randn((low_dim, fn_dim))
    B = torch.randn(fn_dim) + initial_offset
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = (torch.rand((samples,low_dim)) * 2) - 1
        X = x@A + B
        y = fn.evaluate(X)
        
        # Bayesian optimization step
        gp = SingleTaskGP(x, y.reshape(-1,1) * -1)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_model(mll)
        UCB = UpperConfidenceBound(gp, beta=0.1)
        bounds = torch.stack([torch.zeros(low_dim) - 5, torch.zeros(low_dim) + 5])
        candidate, acq_value = optimize_acqf(
            UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
        )
        
        shift_point = candidate@A + B  

        # Update
        if twist_subspace:
            A = torch.concat([shift_point * twist_step_size, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
        else:
            # Else, reinitialize the transition matrix
            A = torch.randn((low_dim, fn_dim))
            
        if move_subspace:
            if trace:
                print(shift_point.shape)
            B = torch.mean(shift_point * move_step_size, dim = 0)

        # Iterate best found (searching for minimum)
        if min(y) < best_found:
            best_point = X[best_performers_indices[0]]
            best_found = min(y)

        if i%int(max_iter/10) == 0:
            print(f"Step {i} current min/best min found y is:{min(y):.2f}, {best_found:.2f}")
            
#     print(f"Best point found at {best_point}")

In [102]:
best_found = 1e10
A = torch.randn((low_dim, fn_dim))
B = torch.randn(fn_dim) + 5
first_run = True            
#     print(f"Best point found at {best_point}")

In [103]:
i = 0
num_best = 1
num_neighbors = 0
move_subspace = True
move_step_size = 1
restarts = 0
samples = 200
twist_subspace = False
trace = False
max_iter = 200

In [113]:
# Evaluate
x = (torch.rand((samples,low_dim)) * 2) - 1
X = x@A + B
y = fn.evaluate(X)

# Bayesian optimization step
gp = SingleTaskGP(x, y.reshape(-1,1))
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)
UCB = UpperConfidenceBound(gp, beta=0.1)
bounds = torch.stack([torch.zeros(low_dim) - 1, torch.zeros(low_dim) + 1])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

shift_point = candidate@A + B  

# Update
if twist_subspace:
    A = torch.concat([shift_point * twist_step_size, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
else:
    # Else, reinitialize the transition matrix
    A = torch.randn((low_dim, fn_dim))

if move_subspace:
    if trace:
        print(shift_point.shape)
    B = torch.mean(shift_point * move_step_size, dim = 0)

# Iterate best found (searching for minimum)
if min(y) < best_found:
    best_point = X[best_performers_indices[0]]
    best_found = min(y)

if i%int(max_iter/10) == 0:
    print(f"Step {i} current min/best min found y is:{min(y):.2f}, {best_found:.2f}")

Step 0 current min/best min found y is:14.89, 14.49


In [114]:
candidate

tensor([[-0.7324,  1.0000]])

In [115]:
acq_value

tensor(15.3196)

In [116]:
fn.evaluate(candidate@A + B)

tensor([15.2920])

In [108]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 1, num_neighbors = 0,
                               move_subspace = True,
                               move_step_size = 1,
                               restarts = 0)

Step 0 current min/best min found y is:8.97, 8.97


NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.

In [12]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:9.00, 9.00
Current min/best min found y is:4.97, 2.26
Current min/best min found y is:8.06, 2.26
Current min/best min found y is:4.24, 2.26
Current min/best min found y is:8.86, 2.26
Current min/best min found y is:3.14, 2.26
Current min/best min found y is:7.42, 1.96
Current min/best min found y is:7.52, 1.96
Current min/best min found y is:8.82, 1.72
Current min/best min found y is:8.18, 1.72


In [76]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:5.19, 5.19
Current min/best min found y is:3.29, 2.33
Current min/best min found y is:2.81, 2.33
Current min/best min found y is:3.66, 2.24
Current min/best min found y is:2.87, 2.24
Current min/best min found y is:3.12, 2.24
Current min/best min found y is:3.34, 2.24
Current min/best min found y is:3.62, 2.24
Current min/best min found y is:3.26, 2.24
Current min/best min found y is:3.38, 2.24


In [77]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10,
                               restarts = 0)

Current min/best min found y is:5.35, 5.35
Current min/best min found y is:1.54, 1.54
Current min/best min found y is:4.92, 1.54
Current min/best min found y is:3.02, 1.54
Current min/best min found y is:5.04, 1.54
Current min/best min found y is:3.38, 1.54
Current min/best min found y is:4.96, 1.54
Current min/best min found y is:2.83, 1.54
Current min/best min found y is:4.70, 1.54
Current min/best min found y is:3.10, 1.54


In [19]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Current min/best min found y is:21.57, 21.57
Current min/best min found y is:21.57, 3.79
Current min/best min found y is:21.57, 3.32
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 2.80
Current min/best min found y is:21.57, 2.80


In [20]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Current min/best min found y is:21.57, 21.57
Current min/best min found y is:3.76, 2.61
Current min/best min found y is:3.38, 2.61
Current min/best min found y is:3.50, 2.44
Current min/best min found y is:3.86, 2.44
Current min/best min found y is:3.23, 2.44
Current min/best min found y is:3.53, 2.44
Current min/best min found y is:3.56, 2.39
Current min/best min found y is:3.18, 2.39
Current min/best min found y is:3.21, 2.39
